In [218]:
import os
import itertools
from Bio import SeqIO

In [219]:
file_path = '/home/gamran/genome_analysis/Warrior/genome/'
p_file_name = 'DK_0911_v01_p_ctg.fa'
h_file_name = 'DK_0911_v01_h_ctg.fa'

h_ctg_file_loc = file_path + h_file_name
p_file_loc = file_path + p_file_name

output_dir = '/home/gamran/genome_analysis/Warrior/Richard/output/'

In [220]:
# returns the primary contig number to which a haplotig or primary contig belongs,
# e.g. hcontig_003_048 will return 003
# e.g. pcontig_008 will return 008

def getPCtgNum(ctg):
    return ctg[8:11]

In [221]:
h_ctgs = [h for h in SeqIO.parse(h_ctg_file_loc, 'fasta')]
p_ctgs = [p for p in SeqIO.parse(p_ctg_file_loc, 'fasta')]

In [222]:
# returns True if primary contig (pCtg) has an associated haplotig
def isCtgPaired(pCtgNum, haplotigs):
    for haplotig in haplotigs:
        if getPCtgNum(haplotig.id) == pCtgNum:
            return True
    return False

In [223]:
for seq_record in itertools.islice(p_ctgs, 1):
    print(seq_record.id)
    print(repr(seq_record.seq))
    print(len(seq_record))
    print(getPCtgNum(seq_record.id))

pcontig_000
Seq('TGTTGTCTATTTTTACTCGTCATTCATGTAAAATGATGTATTTCTTCATTCCTG...gtt', SingleLetterAlphabet())
4476905
000


In [229]:
p_with_h = []
p_without_h = []

for pContig in p_ctgs:
    pCtgNumber = getPCtgNum(pContig.id)
    pContigFileName = p_file_name[:-3] + "_" + getPCtgNum(pContig.id) + ".fa"
    if isCtgPaired(pCtgNumber, h_ctgs):
        p_with_h.append(pContig)
        SeqIO.write(pContig, output_dir + 'pwh/' + pContigFileName, 'fasta')
    else:
        p_without_h.append(pContig)
        SeqIO.write(pContig, output_dir + 'pwoh/' + pContigFileName, 'fasta')

assert len(p_with_h) + len(p_without_h) == len(p_ctgs)

In [228]:
SeqIO.write(p_with_h, output_dir + 'pwh/' + p_file_name[:-3] + '_pwh.fa', 'fasta')
SeqIO.write(p_without_h, output_dir + 'pwoh/' + p_file_name[:-3] + '_pwoh.fa', 'fasta')

13

In [226]:
bash_script_q="ph_ctg_qmapping.sh"
bash_script_g="ph_ctg_gmapping.sh"
outfq = open(output_dir + bash_script_q, 'w')
outfq.write('#!/bin/bash\n')


12